# Temas vistos el 7 de diciembre del 2022
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danielroa98/Postgrad-ML/blob/main/Module-1/07_Dec_2022_lec_09_M1G6P_DRG.ipynb)

# Library imports

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
emp = pd.read_csv('./Datasets/employees_2.csv')
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,8/6/1993,12:42 PM,97308,6.945,True,Marketing
1,Thomas,Male,3/31/1996,6:53 AM,61933,4.170,True,NaN
2,Maria,Female,4/23/1993,11:17 AM,130590,11.858,False,Finance
3,Jerry,Male,3/4/2005,1:00 PM,138705,9.340,True,Finance
4,Larry,Male,1/24/1998,4:47 PM,101004,1.389,True,Client Services


# Continuation from the last [lecture](./05_Dec_2022_lec_09_M1G6P_DRG.ipynb)

## Calculo de variables nuevas a partir de los valores de _diversas_ columnas

Calcular un incremento en el bono (Bonus %) basado en las siguientes condiciones:
1. Si el salario es menor a $90,000 y el equipo es de Marketing --> se hace un incremento del 10%
2. Si el salario es mayor o igual a $90,000 y el equipo es de Finanzas --> se hace un incremento del 15%
3. En otro caso, se deja el bono igual

In [3]:
def new_bonus(employee_row):
    if employee_row['Salary'] < 90000 and employee_row['Team'] == 'Marketing':
        return employee_row['Bonus %']*1.1
    elif employee_row['Salary'] >= 90000 and employee_row['Team'] == 'Finance':
        return employee_row['Bonus %']*1.15
    else:
        return employee_row['Bonus %']


Antes hubiera sido

```python

new_column = Series.apply(function)

```

Ahora se aplica como se ve a continuación:

In [4]:
# Además de poner 'columns' en el axis también se podría poner un 1
emp.apply(new_bonus, axis='columns')

0       6.9450
1       4.1700
2      13.6367
3      10.7410
4       1.3890
        ...   
995    16.6550
996    19.6750
997     1.4210
998    11.9850
999    10.1690
Length: 1000, dtype: float64

In [5]:
emp['New_Bonus'] = emp.apply(new_bonus, axis='columns')
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team,New_Bonus
0,Douglas,Male,8/6/1993,12:42 PM,97308,6.945,True,Marketing,6.9450
1,Thomas,Male,3/31/1996,6:53 AM,61933,4.170,True,NaN,4.1700
2,Maria,Female,4/23/1993,11:17 AM,130590,11.858,False,Finance,13.6367
3,Jerry,Male,3/4/2005,1:00 PM,138705,9.340,True,Finance,10.7410
4,Larry,Male,1/24/1998,4:47 PM,101004,1.389,True,Client Services,1.3890


## Método `copy()`

Probablemente no se quieren sobrescribir los valores de un DataFrame, por lo tanto se crea una copia.

In [6]:
names = emp['First Name'].copy() # La columna original del DataFrame no cambia.
names.head()

0    Douglas
1     Thomas
2      Maria
3      Jerry
4      Larry
Name: First Name, dtype: object

Como es una serie, se pueden obtener datos sin tener que utilizar `loc[]` ni `iloc[]`

In [7]:
names[1:6]

1    Thomas
2     Maria
3     Jerry
4     Larry
5    Dennis
Name: First Name, dtype: object

## `groupby` Object

1. Se crean grupos basados en valores comunes de ciertas columnas.
2. Ideal para agrupar columnas que tienen valores repetidos.

In [8]:
# Este se convierte en un 'groupby' object
teams = emp.groupby('Team')
## el argumento es un string o una lista de strings
## va a hacer un loop a través de las filas que tengan cada uno de los valores de la columna Team


In [9]:
type(teams)

pandas.core.groupby.generic.DataFrameGroupBy

In [10]:
len(teams)

10

In [11]:
emp['Team'].value_counts()

Client Services         106
Finance                 102
Business Development    101
Marketing                98
Product                  95
Sales                    94
Engineering              92
Human Resources          91
Distribution             90
Legal                    88
Name: Team, dtype: int64

In [12]:
emp['Team'].nunique()

10

In [14]:
emp['Team'].nunique(dropna=False)

11

In [15]:
emp['Team'].unique()

array(['Marketing', nan, 'Finance', 'Client Services', 'Legal', 'Product',
       'Engineering', 'Business Development', 'Human Resources', 'Sales',
       'Distribution'], dtype=object)

In [16]:
# Se quiere ver el tamaño de cada equipo
teams.size()

Team
Business Development    101
Client Services         106
Distribution             90
Engineering              92
Finance                 102
Human Resources          91
Legal                    88
Marketing                98
Product                  95
Sales                    94
dtype: int64

### Métodos para `groupby()`

`first()` da el primer registro dentro de cada grupo.

- Por default se ordena alfabeticamente.

In [17]:
teams.first()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,New_Bonus
Team,,,,,,,,
Business Development,Frances,Female,8/8/2002,6:51 AM,139852,7.524,True,7.5240
Client Services,Larry,Male,1/24/1998,4:47 PM,101004,1.389,True,1.3890
Distribution,Michael,Male,10/10/2008,11:25 AM,99283,2.665,True,2.6650
Engineering,Angela,Female,11/22/2005,6:29 AM,95570,18.523,True,18.5230
Finance,Maria,Female,4/23/1993,11:17 AM,130590,11.858,False,13.6367
Human Resources,Brandon,Male,12/1/1980,1:08 AM,112807,17.492,True,17.4920
Legal,Dennis,Male,4/18/1987,1:35 AM,115163,10.125,False,10.1250
Marketing,Douglas,Male,8/6/1993,12:42 PM,97308,6.945,True,6.9450
Product,Ruby,Female,8/17/1987,4:20 PM,65476,10.012,True,10.0120


In [18]:
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team,New_Bonus
0,Douglas,Male,8/6/1993,12:42 PM,97308,6.945,True,Marketing,6.9450
1,Thomas,Male,3/31/1996,6:53 AM,61933,4.170,True,NaN,4.1700
2,Maria,Female,4/23/1993,11:17 AM,130590,11.858,False,Finance,13.6367
3,Jerry,Male,3/4/2005,1:00 PM,138705,9.340,True,Finance,10.7410
4,Larry,Male,1/24/1998,4:47 PM,101004,1.389,True,Client Services,1.3890


`last()` nos da la última ocurrencia dentro de cada grupo.

In [19]:
teams.last()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,New_Bonus
Team,,,,,,,,
Business Development,Larry,Male,4/20/2013,4:45 PM,60500,11.985,False,11.985
Client Services,Robin,Female,7/24/1987,1:35 PM,100765,10.982,True,10.982
Distribution,Henry,Female,11/23/2014,6:09 AM,132483,16.655,False,16.655
Engineering,Tina,Female,5/15/1997,3:53 PM,56450,19.040,True,19.040
Finance,Phillip,Male,1/31/1984,6:30 AM,42392,19.675,False,19.675
Human Resources,Alice,Female,10/5/2004,9:34 AM,47638,11.209,False,11.209
Legal,Justin,Male,2/10/1991,4:58 PM,38344,3.794,False,3.794
Marketing,George,Male,6/21/2013,5:47 PM,98874,4.479,True,4.479
Product,Russell,Male,5/20/2013,12:39 PM,96914,1.421,False,1.421


Un atributo del objeto `groupby` --> atributo groups

In [21]:
teams.groups
## Los índices regresados son del DataFrame original
### ver el siguiente bloque como referencia

{'Business Development': [9, 33, 36, 38, 41, 48, 61, 64, 66, 86, 92, 99, 111, 114, 118, 120, 129, 134, 151, 155, 164, 180, 182, 203, 249, 251, 275, 279, 280, 306, 317, 324, 344, 368, 370, 401, 406, 411, 414, 419, 441, 473, 477, 478, 481, 487, 492, 498, 509, 511, 536, 537, 583, 590, 594, 596, 603, 638, 648, 666, 673, 716, 721, 733, 735, 738, 745, 746, 747, 748, 752, 772, 779, 788, 802, 810, 822, 834, 841, 852, 859, 869, 872, 893, 895, 898, 911, 916, 922, 923, 927, 928, 933, 936, 949, 950, 959, 960, 971, 973, ...], 'Client Services': [4, 18, 22, 24, 25, 28, 34, 37, 39, 47, 70, 82, 85, 95, 102, 128, 133, 145, 152, 163, 174, 184, 196, 197, 206, 207, 212, 215, 217, 227, 234, 244, 254, 265, 282, 284, 287, 291, 292, 303, 315, 325, 330, 333, 334, 359, 366, 385, 387, 389, 392, 408, 423, 432, 440, 448, 472, 489, 497, 502, 506, 524, 529, 543, 554, 560, 564, 569, 574, 577, 587, 605, 606, 621, 631, 639, 651, 655, 658, 670, 677, 679, 694, 728, 729, 742, 751, 755, 767, 782, 796, 845, 866, 877, 879, 9

In [26]:
emp.iloc[[9, 33, 36, 38, 41]]

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team,New_Bonus
9,Frances,Female,8/8/2002,6:51 AM,139852,7.524,True,Business Development,7.524
33,Jean,Female,12/18/1993,9:07 AM,119082,16.180,False,Business Development,16.180
36,Rachel,Female,2/16/2009,8:47 PM,142032,12.599,False,Business Development,12.599
38,Stephanie,Female,9/13/1986,1:52 AM,36844,5.574,True,Business Development,5.574
41,Christine,NaN,6/28/2015,1:08 AM,66582,11.308,True,Business Development,11.308


In [27]:
teams.loc['Client Services'] ## DataFrame de tipo groupby no tiene atributo 'loc'

AttributeError: 'DataFrameGroupBy' object has no attribute 'loc'

### Método `get_group()`

Recordar que: la variable `teams` es la variable donde guardamos nuestro `groupby` object.

In [28]:
teams.get_group('Finance')

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team,New_Bonus
2,Maria,Female,4/23/1993,11:17 AM,130590,11.858,False,Finance,13.63670
3,Jerry,Male,3/4/2005,1:00 PM,138705,9.340,True,Finance,10.74100
7,NaN,Female,7/20/2015,10:43 AM,45906,11.598,NaN,Finance,11.59800
14,Kimberly,Female,1/14/1999,7:13 AM,41426,14.543,True,Finance,14.54300
46,Bruce,Male,11/28/2009,10:47 PM,114796,6.796,False,Finance,7.81540
...,...,...,...,...,...,...,...,...,...
907,Elizabeth,Female,7/27/1998,11:12 AM,137144,10.081,False,Finance,11.59315
954,Joe,Male,1/19/1980,4:06 PM,119667,1.148,True,Finance,1.32020
987,Gloria,Female,12/8/2014,5:08 AM,136709,10.331,True,Finance,11.88065
992,Anthony,Male,10/16/2011,8:35 AM,112769,11.625,True,Finance,13.36875


Antes se hubiera escrito algo así:

In [29]:
emp[emp['Team'] == 'Finance']

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team,New_Bonus
2,Maria,Female,4/23/1993,11:17 AM,130590,11.858,False,Finance,13.63670
3,Jerry,Male,3/4/2005,1:00 PM,138705,9.340,True,Finance,10.74100
7,NaN,Female,7/20/2015,10:43 AM,45906,11.598,NaN,Finance,11.59800
14,Kimberly,Female,1/14/1999,7:13 AM,41426,14.543,True,Finance,14.54300
46,Bruce,Male,11/28/2009,10:47 PM,114796,6.796,False,Finance,7.81540
...,...,...,...,...,...,...,...,...,...
907,Elizabeth,Female,7/27/1998,11:12 AM,137144,10.081,False,Finance,11.59315
954,Joe,Male,1/19/1980,4:06 PM,119667,1.148,True,Finance,1.32020
987,Gloria,Female,12/8/2014,5:08 AM,136709,10.331,True,Finance,11.88065
992,Anthony,Male,10/16/2011,8:35 AM,112769,11.625,True,Finance,13.36875


### Otros métodos para objetos `groupby()`

Lo que hace `sum()` es detectar valores numéricos y hace sumas para cada grupo.

In [30]:
teams.sum()

/var/folders/6b/ddjkk4l908n5cwpnvhllb3fc0000gn/T/ipykernel_16338/871006991.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  teams.sum()


,Salary,Bonus %,New_Bonus
Team,,,
Business Development,9278498,1067.810,1067.8100
Client Services,9351789,1112.481,1112.4810
Distribution,7965042,865.408,865.4080
Engineering,8672766,962.595,962.5950
Finance,9406387,1039.061,1111.3562
Human Resources,8275952,909.443,909.4430
Legal,7858718,908.409,908.4090
Marketing,8862688,1014.638,1068.8044
Product,8423223,930.191,930.1910


Esto se puede manejar como si fuese un DataFrame, por lo tanto se puede invocar por columnas.

In [31]:
teams.sum()['Salary']

/var/folders/6b/ddjkk4l908n5cwpnvhllb3fc0000gn/T/ipykernel_16338/700647215.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  teams.sum()['Salary']


Team
Business Development    9278498
Client Services         9351789
Distribution            7965042
Engineering             8672766
Finance                 9406387
Human Resources         8275952
Legal                   7858718
Marketing               8862688
Product                 8423223
Sales                   8664303
Name: Salary, dtype: int64

Se quiere ver la suma del salario del equipo de 'Marketing'

In [32]:
teams.get_group('Marketing')['Salary'].sum()
## df['col'], donde df = teams.get_group('Marketing')['Salary'].sum()

8862688

In [34]:
teams.mean()

/var/folders/6b/ddjkk4l908n5cwpnvhllb3fc0000gn/T/ipykernel_16338/3205509325.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  teams.mean()


,Salary,Bonus %,New_Bonus
Team,,,
Business Development,91866.316832,10.572376,10.572376
Client Services,88224.424528,10.495104,10.495104
Distribution,88500.466667,9.615644,9.615644
Engineering,94269.195652,10.462989,10.462989
Finance,92219.480392,10.186873,10.895649
Human Resources,90944.527473,9.993879,9.993879
Legal,89303.613636,10.322830,10.322830
Marketing,90435.591837,10.353449,10.906167
Product,88665.505263,9.791484,9.791484


Si se quiere agrupar por más de una variable --> se le pasa una lista

In [35]:
emp.head(2)

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team,New_Bonus
0,Douglas,Male,8/6/1993,12:42 PM,97308,6.945,True,Marketing,6.945
1,Thomas,Male,3/31/1996,6:53 AM,61933,4.170,True,NaN,4.170


In [38]:
team_mgmt = emp.groupby(['Team','Senior Management'], dropna=False)


In [39]:
team_mgmt.size()
## Se puede ver como una tabla dinámica en Excel.

Team                  Senior Management
Business Development  False                45
                      True                 54
                      NaN                   2
Client Services       False                58
                      True                 42
                      NaN                   6
Distribution          False                39
                      True                 38
                      NaN                  13
Engineering           False                37
                      True                 49
                      NaN                   6
Finance               False                51
                      True                 46
                      NaN                   5
Human Resources       False                40
                      True                 45
                      NaN                   6
Legal                 False                45
                      True                 41
                      NaN               

Esto se puede convertir a un _DataFrame_ debido a que es una serie multi-índice.

In [44]:
emp2 = pd.DataFrame(team_mgmt.size(), columns=['Counting'])
emp2.head()

Counting
Team                 Senior Management          
Business Development False                    45
                     True                     54
                     NaN                       2
Client Services      False                    58
                     True                     42

In [45]:
emp2.index

MultiIndex([('Business Development', False),
            ('Business Development',  True),
            ('Business Development',   nan),
            (     'Client Services', False),
            (     'Client Services',  True),
            (     'Client Services',   nan),
            (        'Distribution', False),
            (        'Distribution',  True),
            (        'Distribution',   nan),
            (         'Engineering', False),
            (         'Engineering',  True),
            (         'Engineering',   nan),
            (             'Finance', False),
            (             'Finance',  True),
            (             'Finance',   nan),
            (     'Human Resources', False),
            (     'Human Resources',  True),
            (     'Human Resources',   nan),
            (               'Legal', False),
            (               'Legal',  True),
            (               'Legal',   nan),
            (           'Marketing', False),
          

In [43]:
emp2.reset_index(inplace=True)
emp2.head()

,Team,Senior Management,Counting
0,Business Development,False,45
1,Business Development,True,54
2,Business Development,NaN,2
3,Client Services,False,58
4,Client Services,True,42


## Método `agg()`

In [46]:
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team,New_Bonus
0,Douglas,Male,8/6/1993,12:42 PM,97308,6.945,True,Marketing,6.9450
1,Thomas,Male,3/31/1996,6:53 AM,61933,4.170,True,NaN,4.1700
2,Maria,Female,4/23/1993,11:17 AM,130590,11.858,False,Finance,13.6367
3,Jerry,Male,3/4/2005,1:00 PM,138705,9.340,True,Finance,10.7410
4,Larry,Male,1/24/1998,4:47 PM,101004,1.389,True,Client Services,1.3890


In [48]:
emp.pop('New_Bonus')
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,8/6/1993,12:42 PM,97308,6.945,True,Marketing
1,Thomas,Male,3/31/1996,6:53 AM,61933,4.170,True,NaN
2,Maria,Female,4/23/1993,11:17 AM,130590,11.858,False,Finance
3,Jerry,Male,3/4/2005,1:00 PM,138705,9.340,True,Finance
4,Larry,Male,1/24/1998,4:47 PM,101004,1.389,True,Client Services


In [49]:
# Misma operación para "todas"* las columnas
## *No cuenta strings
teams.sum()

/var/folders/6b/ddjkk4l908n5cwpnvhllb3fc0000gn/T/ipykernel_16338/3464800743.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  teams.sum()


,Salary,Bonus %,New_Bonus
Team,,,
Business Development,9278498,1067.810,1067.8100
Client Services,9351789,1112.481,1112.4810
Distribution,7965042,865.408,865.4080
Engineering,8672766,962.595,962.5950
Finance,9406387,1039.061,1111.3562
Human Resources,8275952,909.443,909.4430
Legal,7858718,908.409,908.4090
Marketing,8862688,1014.638,1068.8044
Product,8423223,930.191,930.1910


Para aggregate, se utilizan dictionaries para definir las operaciones y las columnas en las que se llevarán a cabo.

In [50]:
teams.agg({'Salary':'sum','Bonus %':'mean'})

,Salary,Bonus %
Team,,
Business Development,9278498,10.572376
Client Services,9351789,10.495104
Distribution,7965042,9.615644
Engineering,8672766,10.462989
Finance,9406387,10.186873
Human Resources,8275952,9.993879
Legal,7858718,10.322830
Marketing,8862688,10.353449
Product,8423223,9.791484


#### Nota: También funciona para _DataFrames_.

In [51]:
emp.agg({'Salary':'sum','Bonus %':'mean'})

Salary     9.066218e+07
Bonus %    1.020756e+01
dtype: float64

#### Continuation

In [52]:
teams.agg({
    'Salary':['sum','mean'],
    'Bonus %':'mean'
})

Salary                  Bonus %
                          sum          mean       mean
Team                                                  
Business Development  9278498  91866.316832  10.572376
Client Services       9351789  88224.424528  10.495104
Distribution          7965042  88500.466667   9.615644
Engineering           8672766  94269.195652  10.462989
Finance               9406387  92219.480392  10.186873
Human Resources       8275952  90944.527473   9.993879
Legal                 7858718  89303.613636  10.322830
Marketing             8862688  90435.591837  10.353449
Product               8423223  88665.505263   9.791484
Sales                 8664303  92173.436170  10.116915

In [53]:
emp3 = teams.agg({
    'Salary': ['sum', 'mean'],
    'Bonus %': 'mean'
})


In [55]:
emp3.reset_index()

Team   Salary                  Bonus %
                             sum          mean       mean
0  Business Development  9278498  91866.316832  10.572376
1       Client Services  9351789  88224.424528  10.495104
2          Distribution  7965042  88500.466667   9.615644
3           Engineering  8672766  94269.195652  10.462989
4               Finance  9406387  92219.480392  10.186873
5       Human Resources  8275952  90944.527473   9.993879
6                 Legal  7858718  89303.613636  10.322830
7             Marketing  8862688  90435.591837  10.353449
8               Product  8423223  88665.505263   9.791484
9                 Sales  8664303  92173.436170  10.116915

In [58]:
teams.agg(['size','sum','mean'])

/var/folders/6b/ddjkk4l908n5cwpnvhllb3fc0000gn/T/ipykernel_16338/344479231.py:1: FutureWarning: ['First Name', 'Gender', 'Start Date', 'Last Login Time'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  teams.agg(['size','sum','mean'])


Salary                        Bonus %            \
                       size      sum          mean    size       sum   
Team                                                                   
Business Development    101  9278498  91866.316832     101  1067.810   
Client Services         106  9351789  88224.424528     106  1112.481   
Distribution             90  7965042  88500.466667      90   865.408   
Engineering              92  8672766  94269.195652      92   962.595   
Finance                 102  9406387  92219.480392     102  1039.061   
Human Resources          91  8275952  90944.527473      91   909.443   
Legal                    88  7858718  89303.613636      88   908.409   
Marketing                98  8862688  90435.591837      98  1014.638   
Product                  95  8423223  88665.505263      95   930.191   
Sales                    94  8664303  92173.436170      94   950.990   

                                Senior Management               New_Bonus  \
                           mean              size sum      mean      size   
Team                                                                        
Business Development  10.572376               101  54  0.545455       101   
Client Services       10.495104               106  42  0.420000       106   
Distribution           9.615644                90  38  0.493506        90   
Engineering           10.462989                92  49  0.569767        92   
Finance               10.186873               102  46  0.474227       102   
Human Resources        9.993879                91  45  0.529412        91   
Legal                 10.322830                88  41  0.476744        88   
Marketing             10.353449                98  49  0.538462        98   
Product                9.791484                95  38  0.413043        95   
Sales                 10.116915                94  48  0.558140        94   

                                            
                            sum       mean  
Team                                        
Business Development  1067.8100  10.572376  
Client Services       1112.4810  10.495104  
Distribution           865.4080   9.615644  
Engineering            962.5950  10.462989  
Finance               1111.3562  10.895649  
Human Resources        909.4430   9.993879  
Legal                  908.4090  10.322830  
Marketing             1068.8044  10.906167  
Product                930.1910   9.791484  
Sales                  950.9900  10.116915

### Cómo iterar sobre un objeto `groupby`

Queremos un DataFrame con el salario mas alto de cada equipo

In [62]:
df_aux = pd.DataFrame(columns=emp.columns)
df_aux

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team


In [63]:
# Dentro de cada groupby hay un conjunto de "pares", es decir que cada "par" está compuesto por el valor con el que se hizo el groupby al igual que el DataFrame que lo acompaña.
for team, df in teams:
    highest_gp_salary = df.nlargest(1,'Salary')
    df_aux = df_aux.append(highest_gp_salary)

/var/folders/6b/ddjkk4l908n5cwpnvhllb3fc0000gn/T/ipykernel_16338/3005351152.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aux = df_aux.append(highest_gp_salary)
/var/folders/6b/ddjkk4l908n5cwpnvhllb3fc0000gn/T/ipykernel_16338/3005351152.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aux = df_aux.append(highest_gp_salary)
/var/folders/6b/ddjkk4l908n5cwpnvhllb3fc0000gn/T/ipykernel_16338/3005351152.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aux = df_aux.append(highest_gp_salary)
/var/folders/6b/ddjkk4l908n5cwpnvhllb3fc0000gn/T/ipykernel_16338/3005351152.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aux

In [64]:
df_aux

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
721,Harold,Male,4/16/2010,5:13 AM,147417,11.626,True,Business Development
287,Lois,Female,11/9/2011,7:06 AM,147183,9.999,True,Client Services
793,Andrea,Female,7/22/1999,9:25 AM,149105,13.707,True,Distribution
541,Ruby,Female,5/1/1999,3:36 AM,147362,7.851,True,Engineering
644,Katherine,Female,8/13/1996,12:21 AM,149908,18.912,False,Finance
429,Rose,Female,5/28/2015,8:40 AM,149903,5.630,False,Human Resources
981,James,Male,1/15/1993,5:19 PM,148985,19.280,False,Legal
740,Russell,NaN,5/9/2009,11:59 AM,149456,3.533,False,Marketing
828,Cynthia,Female,7/12/2006,8:55 AM,149684,7.864,False,Product
186,NaN,Female,2/23/2005,9:50 PM,149654,1.825,NaN,Sales


# Numpy

Librería que se utiliza principalmente para trabajar con arreglos de números. También cuenta con funciones que permiten trabajar con Álgebra Lineal y Mátrices.

Definir una matriz de 2 renglones por 3 columnas.

In [65]:
arr1 = np.array(
    [[1, 2, 3],
     [4, 5, 6]]
)
arr1


array([[1, 2, 3],
       [4, 5, 6]])

Una de las principales ventajas de Numpy es el poder usar vectorización de funciones aplicadas a un __np.array__, i.e: poder aplicar una función a cada renglón sin la necesidad de usar un loop, lo cual lo hace más eficiente.

In [67]:
%%timeit -n 5
sum([i**2 for i in range(1000)])

262 µs ± 58 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


np.arrange puede ser visto como un _range()_ clásico de Python, pero el _arrange_ puede fraccionar los steps, mientras que en el _range_ solo se puede con valores enteros.

In [68]:
%%timeit -n 5
(np.arange(1000)**2).sum()

The slowest run took 22.26 times longer than the fastest. This could mean that an intermediate result is being cached.
38.4 µs ± 32.2 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [69]:
arr1.shape

(2, 3)

In [70]:
arr1

array([[1, 2, 3],
       [4, 5, 6]])

In [71]:
arr1[0]

array([1, 2, 3])

In [73]:
arr1[1]

array([4, 5, 6])

In [74]:
arr2 = np.array([
    [1, 2, 3, 4],
    [4, 5, 6, 5],
    [7, 8, 9, 0],
    [3, 4, 5, 6]
])
arr2

array([[1, 2, 3, 4],
       [4, 5, 6, 5],
       [7, 8, 9, 0],
       [3, 4, 5, 6]])

In [75]:
arr2.shape

(4, 4)

## Operaciones principales con arreglos de Numpy

In [76]:
a1 = np.array([
    [2,4,6],
    [2,5,1]
])

### Broadcasting operations

La suma de un escalar a cada elemento del array

In [77]:
a1 + 1

array([[3, 5, 7],
       [3, 6, 2]])

También se puede incrementar una variable.

- A diferencia de lo de arriba, al utilizar `+=` se asigna el valor a la variable.

In [78]:
a1 += 1
a1

array([[3, 5, 7],
       [3, 6, 2]])

Elevando al cuadrado cada elemento del array

In [79]:
a1**2

array([[ 9, 25, 49],
       [ 9, 36,  4]])

Multiplicar entrada por entrada.

In [80]:
a1*a1

array([[ 9, 25, 49],
       [ 9, 36,  4]])

Comparación lógica
- Es entrada por entrada

In [81]:
a1 <= 3

array([[ True, False, False],
       [ True, False,  True]])

### `np.arrange` y `np.linspace`

Podemos crear rangos de números [a,b] usando la función `arange`, la cual es la versión análoga de `range`  en Python.

In [83]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

En el `range` clásico de Python solo hay enteros en el _step_, en Numpy si se pueden usar _steps_ fraccionarios.
- El límite superior __no es inclusivo__.

In [84]:
np.arange(1,11,0.5)

array([ 1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  5.5,  6. ,
        6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. , 10.5])

Se puede usar la función `linspace()` cuando deseamso un arreglo de $n$ elementos entre $a$ y $b$ (inclusivo); $a < b$

En `linspace()` se tienen los siguientes valores dentro
`linspace(start, stop, number of buckets)`

In [85]:
a, b = 2, 10
np.linspace(a,b,20)

array([ 2.        ,  2.42105263,  2.84210526,  3.26315789,  3.68421053,
        4.10526316,  4.52631579,  4.94736842,  5.36842105,  5.78947368,
        6.21052632,  6.63157895,  7.05263158,  7.47368421,  7.89473684,
        8.31578947,  8.73684211,  9.15789474,  9.57894737, 10.        ])

### Índices

In [88]:
arr = np.arange(30).reshape(10,3)
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20],
       [21, 22, 23],
       [24, 25, 26],
       [27, 28, 29]])

Seleccionar la primera fila

In [89]:
arr[0]

array([0, 1, 2])

Primer renglón, todas las columnas

In [90]:
arr[0,:]

array([0, 1, 2])

Segunda columna

In [91]:
arr[:,1]

array([ 1,  4,  7, 10, 13, 16, 19, 22, 25, 28])

Todos los renglones, desde la columna especificada

In [92]:
arr[:,1:]

array([[ 1,  2],
       [ 4,  5],
       [ 7,  8],
       [10, 11],
       [13, 14],
       [16, 17],
       [19, 20],
       [22, 23],
       [25, 26],
       [28, 29]])

Extrae la primer y tercer columna

In [93]:
arr[:,[0,2]]

array([[ 0,  2],
       [ 3,  5],
       [ 6,  8],
       [ 9, 11],
       [12, 14],
       [15, 17],
       [18, 20],
       [21, 23],
       [24, 26],
       [27, 29]])

Podemos utilizar índices para encontrar elementos

In [94]:
row, col = 0,2
arr[row, col]

2

Podemos encontrar mas de un elemento (-2, 1) y (1, -1):

In [95]:
rows, cols = [-2,-1], [1,-1]
arr[rows, cols]

array([25, 29])

Equivalente a `arr[[0,-1],_]`

In [96]:
arr[[0,-1]]

array([[ 0,  1,  2],
       [27, 28, 29]])

Se pueden asignar varios valores de la misma dimensión a la que se le hizo la selección

In [97]:
# original
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20],
       [21, 22, 23],
       [24, 25, 26],
       [27, 28, 29]])

In [98]:
arr[[0, -1]] = np.random.randint(-100, -1, size=(2, 3))

In [99]:
# después de ser modificado
arr

array([[-17, -24, -65],
       [  3,   4,   5],
       [  6,   7,   8],
       [  9,  10,  11],
       [ 12,  13,  14],
       [ 15,  16,  17],
       [ 18,  19,  20],
       [ 21,  22,  23],
       [ 24,  25,  26],
       [-44, -92, -94]])